# Análisis de Valores Nulos - Mieles Finales
**Objetivo:** Identificar patrones de datos faltantes por ingenio, mes y temporada

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Cargar Datos

In [ ]:
# Leer archivo
df = pd.read_excel('../data/raw/2025_Base_Miel_Final-_calculos.xlsx', sheet_name='Base calculada')

# Headers en primera fila
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

# Eliminar primera columna si es NaN
if pd.isna(df.columns[0]):
    df = df.iloc[:, 1:]

print(f"Dataset: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"Período: {df['Zafra'].min()} a {df['Zafra'].max()}")
print(f"Ingenios: {df['Ingenio'].nunique()}")

## 2. Resumen de Nulos

In [ ]:
# Columnas identificadoras vs datos
id_cols = ['Zafra', 'NoM', 'Mes', 'Muestra', 'Código_ING', 'Ingenio']
data_cols = [col for col in df.columns if col not in id_cols]

# Contar nulos
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

print(f"\nColumnas con nulos: {len(null_counts)}/{len(df.columns)}")
print(f"Total nulos: {df.isnull().sum().sum()}")
print(f"% del dataset: {(df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100:.2f}%")

## 3. Nulos por Columna

In [ ]:
# Top 10 columnas con más nulos
print("\nTop 10 columnas con nulos:")
for col, count in null_counts.head(10).items():
    pct = (count / len(df)) * 100
    print(f"{col:30s}: {count:3d} ({pct:5.1f}%)")

## 4. Análisis por Ingenio

In [ ]:
# Nulos por ingenio
mill_nulls = []
for mill in df['Ingenio'].unique():
    mill_data = df[df['Ingenio'] == mill]
    total_cells = mill_data.shape[0] * len(data_cols)
    null_cells = mill_data[data_cols].isnull().sum().sum()
    mill_nulls.append({
        'Ingenio': mill,
        'Registros': len(mill_data),
        'Nulos': null_cells,
        'Null %': (null_cells / total_cells) * 100
    })

mill_df = pd.DataFrame(mill_nulls).sort_values('Null %', ascending=False)
print("\nNulos por ingenio:")
print(mill_df.to_string(index=False))

## 5. Filas Completamente Nulas

In [ ]:
# Identificar filas completamente nulas (solo datos, no IDs)
completely_null = []
for idx, row in df.iterrows():
    if row[data_cols].isnull().all():
        completely_null.append({
            'Ingenio': row['Ingenio'],
            'Zafra': row['Zafra'],
            'Mes': row['Mes'],
            'NoM': row['NoM']
        })

if completely_null:
    null_rows_df = pd.DataFrame(completely_null)
    print(f"\nFilas completamente nulas: {len(completely_null)}")
    print(null_rows_df.to_string(index=False))
    
    # Contar por ingenio
    print("\nPor ingenio:")
    print(null_rows_df['Ingenio'].value_counts())
else:
    print("\nNo hay filas completamente nulas")

## 6. Patrones de Nulos Agrupados

In [ ]:
# Detectar grupos de columnas que siempre tienen nulos juntas
print("\nGrupos de nulos detectados:")

# Grupo 1: Viscosidad
visc_nulls = df[df['Viscosidad_25C'].isnull()]
print(f"\nViscosidad (25C y 40C): {len(visc_nulls)} filas")
print(f"  Ingenios: {visc_nulls['Ingenio'].value_counts().to_dict()}")
print(f"  Zafras: {visc_nulls['Zafra'].value_counts().to_dict()}")

# Grupo 2: Cálculos de pérdida
loss_nulls = df[df['kg/t_MF'].isnull()]
print(f"\nPérdidas (kg/t_MF, etc.): {len(loss_nulls)} filas")
print(f"  Ingenios: {loss_nulls['Ingenio'].value_counts().to_dict()}")

# Grupo 3: Análisis químico completo
chem_nulls = df[df['Brix'].isnull()]
print(f"\nAnálisis químico completo: {len(chem_nulls)} filas")
if len(chem_nulls) > 0:
    print(f"  Detalles:")
    for _, row in chem_nulls.iterrows():
        print(f"    {row['Ingenio']} - {row['Mes']} {row['Zafra']}")

## 7. Hallazgos Clave

**Patrón 1: Tulula**
- Mayor cantidad de nulos (verificar si son meses sin producción azucarera)
- Posible producción de ron en esos períodos

**Patrón 2: Viscosidad 2020-2021**
- Posible problema de equipo en primera zafra

**Patrón 3: Mayo (fin de zafra)**
- Muestras incompletas en cierre de temporada

**Siguiente paso:** Definir estrategia de limpieza

## 8. Exportar Reporte

In [ ]:
# Crear reporte de nulos
null_report = null_counts.reset_index()
null_report.columns = ['Columna', 'Nulos']
null_report['%'] = (null_report['Nulos'] / len(df) * 100).round(2)

# Guardar
null_report.to_csv('../reports/null_analysis_report.csv', index=False)
print("\nReporte guardado: reports/null_analysis_report.csv")